<a href="https://colab.research.google.com/github/noushadkhan01/internshala/blob/master/internshal_web_scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
def clean_text(text):
  import string
  punctuations = string.punctuation.replace('-', '').replace("'", '') + '\n'
  text = [i for i in text if i not in punctuations]
  text = ''.join(text)
  text = text.strip()
  return text

In [0]:
def clean_stipend(text):
  import numpy as np
  text = text.lower().strip()
  text = [i for i in text]
  a = ''
  final_text = ''
  for i in text:
    a += i
    try:
      np.float64(a)
      final_text = a
    except:
      break
  if final_text:
    return np.float64(final_text)
  return np.nan

In [0]:
def clean_apply_by(text):
  f = text.find("'")
  text = text[:f] + ' 2020'
  if text == 'Not Provide 2020':
    return np.nan
  return text

In [0]:
def apply_by_to_date(text):
  from datetime import datetime
  a = datetime.strptime(text, '%d %b %Y')
  return a

In [0]:
def get_internshala_internships(url, n = 0):
  import numpy as np
  import pandas as pd
  import requests 
  from bs4 import BeautifulSoup
  all_items = []
  no_internship_text = ":( Sorry, We couldn't find internships matching your requirements."
  while True:
    print(f'n is {n}')
    if n:
      url = url + f'/page-{n}'
    try:
      html = requests.get(url)
      soup = BeautifulSoup(html.content, 'lxml')
      all_internship = soup.find_all('div', class_ = 'individual_internship')
      text = all_internship[0].find('h4').text
      if text == no_internship_text:
        print(no_internship_text)
        break
      l = len(all_internship)
      n2 = 0
      for i in all_internship:
        try:
          id = i.get('internshipid')
          location = i.find_all('a', class_ = 'location_link')[0].text
          link = 'https://internshala.com/' + i.find_all('a')[0].get('href')
          td = i.find_all('table')[0].find_all('td')
          start_date = td[0].text
          duration = td[1].text
          stipend = td[2].text
          posted_on = td[3].text
          apply_by = td[4].text
          items = [id, location, start_date, duration, stipend, posted_on, apply_by]
          items = list(map(clean_text, items))
          items.insert(2, link)
          del id, location, link, start_date, duration, stipend, posted_on, apply_by
          all_items.append(items)
          n2 += 1
        except:
          continue
      n += 1
    except Exception as e:
      print(f'{e.__class__.__name__}:--- {e.args[0]}')
      break
  if len(all_items) == 0:
    return no_internship_text
  df = pd.DataFrame(np.array(all_items), columns = ['id', 'location', 'link', 'start_date', 'duration', 'stipend', 'posted_on', 'apply_by'])
  df.drop_duplicates(inplace = True)
  df['stipend'] = df.stipend.apply(clean_stipend)
  df['apply_by'] = df['apply_by'].apply(clean_apply_by)
  df['apply_by'] = df['apply_by'].apply(apply_by_to_date)
  df.dropna(inplace = True)
  return df

In [0]:
url = 'https://internshala.com/internships/work-from-home-data%20science,machine%20learning-jobs'

In [112]:
work_from_home = get_internshala_internships(url)

n is 0
n is 1
n is 2
n is 3
:( Sorry, We couldn't find internships matching your requirements.


In [113]:
work_from_home.head()

,id,location,link,start_date,duration,stipend,posted_on,apply_by
0,886861,Work From Home,https://internshala.com//internship/detail/dee...,Immediately,6 Months,2000.0,26 Mar'20,2020-04-23
1,884133,Work From Home,https://internshala.com//internship/detail/dat...,20 Apr - 30 Apr'20,4 Months,1000.0,20 Mar'20,2020-04-17
2,877851,Work From Home,https://internshala.com//internship/detail/tea...,Immediately,6 Months,15000.0,15 Mar'20,2020-04-13
3,886851,Work From Home,https://internshala.com//internship/detail/dia...,Immediately,1 Month,2000.0,31 Mar'20,2020-04-28
5,888678,Work From Home,https://internshala.com//internship/detail/bus...,Immediately,3 Months,5000.0,31 Mar'20,2020-04-28


In [115]:
work_from_home.shape

(46, 8)

In [0]:
work_from_home.sort_values(by = 'apply_by', inplace = True)

In [117]:
work_from_home.head(2)

,id,location,link,start_date,duration,stipend,posted_on,apply_by
19,887271,Work From Home,https://internshala.com//internship/detail/art...,Immediately,4 Months,5000.0,26 Mar'20,2020-04-09
85,875255,Work From Home,https://internshala.com//internship/detail/bus...,Immediately,2 Months,5000.0,11 Mar'20,2020-04-09


In [118]:
useful_data = work_from_home[work_from_home.apply_by <= '2020-04-10']
useful_data.shape

(4, 8)

In [121]:
useful_data.sort_values(by = 'stipend', ascending = False, inplace = True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [122]:
useful_data

,id,location,link,start_date,duration,stipend,posted_on,apply_by
84,869810,Work From Home,https://internshala.com//internship/detail/dat...,Immediately,3 Months,25000.0,12 Mar'20,2020-04-10
19,887271,Work From Home,https://internshala.com//internship/detail/art...,Immediately,4 Months,5000.0,26 Mar'20,2020-04-09
85,875255,Work From Home,https://internshala.com//internship/detail/bus...,Immediately,2 Months,5000.0,11 Mar'20,2020-04-09
82,876162,Work From Home,https://internshala.com//internship/detail/web...,Immediately,2 Months,1000.0,12 Mar'20,2020-04-10


In [125]:
print(useful_data.link.values)

['https://internshala.com//internship/detail/data-analytics-work-from-home-job-internship-at-stoic-investment-managers1583232868'
 'https://internshala.com//internship/detail/artificial-intelligence-machine-learning-image-processing-work-from-home-job-internship-at-inventgrid-india-private-limited1585241191'
 'https://internshala.com//internship/detail/business-analytics-work-from-home-job-internship-at-stirring-minds1583753814'
 'https://internshala.com//internship/detail/web-scrapping-and-data-analytics-work-from-home-job-internship-at-heycloudy1583905948']
